In [1]:
pip install transformers    ##Instalar transformers


In [2]:
##Librerias requeridas
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
#Cargar modelo preentrenado y toquenizador
tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/dehatebert-mono-spanish")
model = AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/dehatebert-mono-spanish")
#cargar el dataset
dataset_path = "dataset_etiquetado.txt"
#leer el archivo y dividirlo en etiquetas
with open(dataset_path, "r", encoding="utf-8") as file:
    lines = file.readlines()

labels = [int(line.split('\t')[0]) for line in lines]
texts = [line.split('\t')[1].strip() for line in lines]

#Se divide el dataset en conjuntos de entrenamiento y pruebas
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42 ##Se escoge 20% para pruebas
)

#Tokenizar los textos
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

#se crea la clase CustomDataset que hereda de Dataset para que los datos puedan ser usados por PyTorch
class CustomDataset(Dataset):
  ##Cargando los textos y etiquetas tokenizadas
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
  ##Esta funcion accede a elementos individuales usando el indice idx
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx]) #Se añade la entrada labels para el tensor de Pytorch
        return item

    def __len__(self): ##Devuelve la longitud del conjunto de datos para determinar cuantas iteraciones para cada epoca de entrenamiento
        return len(self.labels)

##Se crea instancias de CustomDataset una para entrenamiento y otra para prueba
train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

#Configuracion del entrenamiento
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #Se selecciona que dispositivo hara el entrenamiento
model.to(device)##se mueve el modelo al mismo espacio de los datos

#Se crea un DataLoader de  pytorch, usara los datos de entrenamiento en lotes de 8 muestras a lavez ademas de mezclarlas
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

#Configurar el optimizador y el scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)  ##Se usa algoritmo AdamW con la taza de aprendizaje
num_epochs = 3 ##Numero de epocas de entrenamiento
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

#Entrenamiento
for epoch in range(num_epochs): ##bucle que entrenara en cantidad de epocas establecidas
    model.train() ##Establecer el modelo en modo de entrenamiento
    for batch in train_loader:
        inputs = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    scheduler.step()

#Evaluar el modelo
model.eval()    ##Cambia el modo del modelo a evaluacion
##Se crea un Dataloader de pytorch, se evaluara en lotes de 8 muestras
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)
#iniciacion de variables.
total_correct = 0
total_samples = 0

with torch.no_grad(): #Se evaluara sin aprender
    for batch in test_loader:
        inputs = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        #Obtiene la salida de logit(puntuaciones) del modelo
        predictions = torch.argmax(logits, dim=1)
        total_correct += torch.sum(predictions == inputs["labels"])
        total_samples += len(inputs["labels"])

accuracy = total_correct.item() / total_samples ##Calcular precision
print(f"Accuracy on test set: {accuracy * 100:.2f}%")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Accuracy on test set: 89.47%


In [4]:
#Guardar los pesos del modelo después del entrenamiento
model.save_pretrained("/content/modelo")
